# Бустинг

На этом уроке вы продолжете работать с датасетом, с котором вы уже работали. Однако сегодня вы запустите на нем более крутые алгоритмы машинного обучения. Удачи!

In [25]:
import pandas as pd

In [26]:
# Данные библиотеки могут быть не установленны. 
# Установите их через pip install/ conda install
import xgboost
import lightgbm as lgb
from catboost import CatBoostClassifier

In [27]:
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier

In [28]:
data = pd.read_csv('dota2_skill_train.csv', index_col='id')
data.shape

(99871, 57)

In [29]:
data.head()

,skilled,player_team,winner_team,duration,pre_game_duration,first_blood_time,first_blood_claimed,hero_id,hero_pick_order,leaver_status,...,avg_deaths_x16,avg_assists_x16,avg_gpm_x16,avg_xpm_x16,best_kills_x16,best_assists_x16,best_gpm_x16,best_xpm_x16,win_streak,best_win_streak
id,,,,,,,,,,,,,,,,,,,,,
7,1,dire,dire,2140,90,129,0,90,9,0,...,8,15,352,430,10,30,551,745,2,5
8,0,radiant,radiant,2138,90,174,0,5,5,0,...,9,19,294,425,13,37,445,717,2,12
12,0,radiant,radiant,3547,90,360,0,81,7,0,...,7,9,493,543,23,18,691,762,3,3
13,1,dire,radiant,1878,90,28,0,74,9,0,...,7,12,515,583,25,34,869,935,0,6
14,1,dire,radiant,2232,90,129,0,14,6,0,...,10,16,337,452,34,43,672,797,0,9


In [30]:
# Кодируем категориальные фичи 
data['player_team'] = data['player_team'].map({'dire':0, 'radiant':1})
data['winner_team'] = data['winner_team'].map({'dire':0, 'radiant':1})

## Вспомнить всё 

In [31]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('skilled', axis=1), data.skilled, random_state=42)

In [32]:
tree_6 = DecisionTreeClassifier(max_depth=6)
tree_6.fit(X_train, y_train)

pred = tree_6.predict(X_test)
accuracy_score(y_test, pred)

0.662528035885934

In [33]:
print('ROC-AUC: ', roc_auc_score(y_test, pred))
print(confusion_matrix(y_test, pred))

ROC-AUC:  0.6117802842521022
[[12883  2239]
 [ 6187  3659]]


## Построим бустинговые модели и сравним результаты 

## Xgboost 

In [35]:
'''
Мы можем добавлять параметры модели с помощью словаря параметров. 
Это удобно т.к. в бустинговых моделях параметров намного больше, чем в линейных

'''


params = { 
    'eta': 0.07, #то же самое, что и learning rate
    'max_depth': 8, 
    'objective': 'binary:logistic',
    'eval_metric': 'Logloss',
    'seed': 7}


model = xgboost.XGBClassifier(num_round = 100)
model.fit(X_train, y_train,  eval_set=[(X_test, y_test)])

[21:47:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { num_round } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:47:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.65382
[1]	validation_0-logloss:0.62881
[2]	validation_0-logloss:0.61279
[3]	validation_0-logloss:0.60252
[4]	validation_0-logloss:0.59471
[5]	validation_0-logloss:0.58604
[6]	validation_0-logloss:0.58129
[7]	validation_0-logloss:0.57764
[8]	validation_0-logloss:0.57200
[9]	va

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, num_round=100,
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

#### Построим confusion matrix и roc-auc 
Важно: бустинговые модели, как правило, выдают вероятность принадлежности к классу.

In [36]:
# В данном случае достаточно слова predict
y_predict_proba = model.predict(X_test)

In [37]:
print('ROC-AUC: ', roc_auc_score(y_test, y_predict_proba))
print(confusion_matrix(y_test, y_predict_proba))

ROC-AUC:  0.7294444214746535
[[12559  2563]
 [ 3659  6187]]


## Catboost 

Все параметры смотреть здесь:
https://tech.yandex.com/catboost/doc/dg/concepts/parameter-tuning-docpage/

Мы настроим только несколько параметров:
 - количество итераций
 - глубину деревьев
 - метрику качества
 - функцию потерь
 - random seed, чтобы результаты моделей были сравнимы
 
В каждой настроенной модели мы будем использовать одни и те же параметры, опять же, чтобы результаты были похожи и объективны

In [38]:
model = CatBoostClassifier(iterations=100,
                           learning_rate=0.07, 
                           depth=8, 
                           eval_metric = 'AUC',
                           loss_function='Logloss',
                          random_state=7)
model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)

0:	test: 0.6860599	best: 0.6860599 (0)	total: 75.7ms	remaining: 7.5s
1:	test: 0.6973464	best: 0.6973464 (1)	total: 135ms	remaining: 6.63s
2:	test: 0.7022035	best: 0.7022035 (2)	total: 193ms	remaining: 6.25s
3:	test: 0.7079006	best: 0.7079006 (3)	total: 248ms	remaining: 5.95s
4:	test: 0.7112800	best: 0.7112800 (4)	total: 305ms	remaining: 5.79s
5:	test: 0.7169482	best: 0.7169482 (5)	total: 361ms	remaining: 5.65s
6:	test: 0.7168093	best: 0.7169482 (5)	total: 416ms	remaining: 5.52s
7:	test: 0.7185901	best: 0.7185901 (7)	total: 474ms	remaining: 5.45s
8:	test: 0.7199292	best: 0.7199292 (8)	total: 537ms	remaining: 5.43s
9:	test: 0.7229102	best: 0.7229102 (9)	total: 592ms	remaining: 5.33s
10:	test: 0.7249060	best: 0.7249060 (10)	total: 649ms	remaining: 5.25s
11:	test: 0.7264539	best: 0.7264539 (11)	total: 707ms	remaining: 5.18s
12:	test: 0.7280701	best: 0.7280701 (12)	total: 778ms	remaining: 5.2s
13:	test: 0.7298126	best: 0.7298126 (13)	total: 839ms	remaining: 5.16s
14:	test: 0.7324075	best: 0

#### Построим confusion matrix и roc-auc 

In [39]:
y_predict_proba = model.predict(X_test)

In [40]:
print('ROC-AUC: ', roc_auc_score(y_test, y_predict_proba))
print(confusion_matrix(y_test, y_predict_proba))

ROC-AUC:  0.6854641629218521
[[12777  2345]
 [ 4667  5179]]


## LightGBM 

Полезно посмотреть документацию:https://lightgbm.readthedocs.io/en/latest/Parameters.html.
Это позволит вам поиграться с параметрами.

In [41]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


params = {'max_depth':8, #глубина деревьев
    'boosting_type': 'gbdt', #модель для бустинга, в данном случае - gradient bosting decision tree
    'objective': 'binary', #задача
    'metric': ['auc'], #метрика качества
    'learning_rate': 0.07, #
     'seed': 7}

In [42]:
gbm = lgb.train(params, lgb_train,
            num_boost_round=100, #количество построений-улучшений алгоритма
            valid_sets=[lgb_train, lgb_eval],
            early_stopping_rounds=10 #после этого количества одинаковых результатов, алгоритм досрочно закончит обучение
               )

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 29940, number of negative: 44963
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015757 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7462
[LightGBM] [Info] Number of data points in the train set: 74903, number of used features: 56
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.399717 -> initscore=-0.406645
[LightGBM] [Info] Start training from score -0.406645
[1]	training's auc: 0.701242	valid_1's auc: 0.693456
Training until validation scores don't improve for 10 rounds

#### Оценим модель 

In [43]:
y_predict_proba = gbm.predict(X_test)

In [44]:
print('ROC-AUC: ', roc_auc_score(y_test, y_predict_proba))

ROC-AUC:  0.7919634504687892


In [45]:
for i in range(len(y_predict_proba)):
    if y_predict_proba[i]<0.5:
        y_predict_proba[i]=0
    else:
        y_predict_proba[i] = 1

In [46]:
confusion_matrix(y_test,y_predict_proba)

array([[12701,  2421],
       [ 4297,  5549]], dtype=int64)